<a href="https://colab.research.google.com/github/Anirudh-020505/Solar-Energy-Forecasting/blob/main/solarForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# import pandas as pd

# # 1. Mount the Drive (Follow the popup instructions)
# drive.mount('/content/drive')

# # 2. Define the path (Since they added a shortcut, the path is the same for everyone!)
# # Note: Ensure the folder name in My Drive is exactly 'Solar_Project_Data'
# path = '/content/drive/My Drive/Solar-ForeCast-Resource/'

# # 3. Load the Data
# gen_df = pd.read_csv(path + 'spg.csv')

# print("Data Loaded Successfully!")
# print(gen_df.head())

In [2]:
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("stucom/solar-energy-power-generation-dataset")

print("Path to dataset files:", path)

100%|██████████| 201k/201k [00:00<00:00, 29.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/stucom/solar-energy-power-generation-dataset/versions/1


In [7]:
import os
import pandas as pd
import numpy as np

print(os.listdir(path))   # shows files

df = pd.read_csv(path + "/spg.csv")   # correct file name
df.head()


['spg.csv']


,temperature_2_m_above_gnd,relative_humidity_2_m_above_gnd,mean_sea_level_pressure_MSL,total_precipitation_sfc,snowfall_amount_sfc,total_cloud_cover_sfc,high_cloud_cover_high_cld_lay,medium_cloud_cover_mid_cld_lay,low_cloud_cover_low_cld_lay,shortwave_radiation_backwards_sfc,...,wind_direction_10_m_above_gnd,wind_speed_80_m_above_gnd,wind_direction_80_m_above_gnd,wind_speed_900_mb,wind_direction_900_mb,wind_gust_10_m_above_gnd,angle_of_incidence,zenith,azimuth,generated_power_kw
0,2.17,31,1035.0,0.0,0.0,0.0,0,0,0,0.00,...,312.71,9.36,22.62,6.62,337.62,24.48,58.753108,83.237322,128.33543,454.10095
1,2.31,27,1035.1,0.0,0.0,0.0,0,0,0,1.78,...,294.78,5.99,32.74,4.61,321.34,21.96,45.408585,75.143041,139.65530,1411.99940
2,3.65,33,1035.4,0.0,0.0,0.0,0,0,0,108.58,...,270.00,3.89,56.31,3.76,286.70,14.04,32.848282,68.820648,152.53769,2214.84930
3,5.82,30,1035.4,0.0,0.0,0.0,0,0,0,258.10,...,323.13,3.55,23.96,3.08,339.44,19.80,22.699288,64.883536,166.90159,2527.60920
4,7.73,27,1034.4,0.0,0.0,0.0,0,0,0,375.58,...,10.01,6.76,25.20,6.62,22.38,16.56,19.199908,63.795208,182.13526,2640.20340


In [8]:
df = df.drop_duplicates()
df.columns = df.columns.str.lower().str.strip()

In [9]:
if "wind_direction_10_m_above_gnd" in df.columns:
    df["wind_dir_10_sin"] = np.sin(np.deg2rad(df["wind_direction_10_m_above_gnd"]))
    df["wind_dir_10_cos"] = np.cos(np.deg2rad(df["wind_direction_10_m_above_gnd"]))
    df = df.drop("wind_direction_10_m_above_gnd", axis=1)

In [10]:
numeric_cols = df.select_dtypes(include=np.number).columns

Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[numeric_cols] < (Q1 - 1.5 * IQR)) |
          (df[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [11]:
df = df.ffill()

In [12]:
X = df.drop("generated_power_kw", axis=1)
y = df["generated_power_kw"]

print("Cleaned Shape:", df.shape)
print(df.head())

Cleaned Shape: (2362, 22)
   temperature_2_m_above_gnd  relative_humidity_2_m_above_gnd  \
0                       2.17                               31   
1                       2.31                               27   
2                       3.65                               33   
3                       5.82                               30   
4                       7.73                               27   

   mean_sea_level_pressure_msl  total_precipitation_sfc  snowfall_amount_sfc  \
0                       1035.0                      0.0                  0.0   
1                       1035.1                      0.0                  0.0   
2                       1035.4                      0.0                  0.0   
3                       1035.4                      0.0                  0.0   
4                       1034.4                      0.0                  0.0   

   total_cloud_cover_sfc  high_cloud_cover_high_cld_lay  \
0                    0.0                   